# Este código toma el fichero de transacciones de 3 meses
(y agrega por Group by tokens y accounts, para obtener un sumario de los Traders más rentables

eliminando aquellos con un inventario negativo)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Define el directorio y el nombre del archivo
directory = 'hist_input/'
filename = 'top_traders_AGREG_Q_c.csv'

# Construye la ruta completa al archivo
file_path = directory + filename

In [2]:
# Función para procesar el DataFrame por cada 'pair_id_token'
def process_data_for_each_token(df):
    summaries = []
    for token in df['pair_id_token'].unique():
        df_token = df[df['pair_id_token'] == token]
        # Calcula 'buy_Q' y 'sell_Q'
        df_token['buy_Q'] = df_token.apply(lambda row: row['tokens_Q'] if row['type'] == 'BUY' else 0, axis=1)
        df_token['sell_Q'] = df_token.apply(lambda row: row['tokens_Q'] if row['type'] == 'SELL' else 0, axis=1)
        # Redondea 'buy_Q' y 'sell_Q'
        df_token['buy_Q'] = round(df_token['buy_Q'] / 1000, 0)
        df_token['sell_Q'] = round(df_token['sell_Q'] / 1000, 0)
        
        account_summary = df_token.groupby('account').agg(
            total_buy=('buy', 'sum'),
            total_sell=('sell', 'sum'),
            total_buy_Q=('buy_Q', 'sum'),
            total_sell_Q=('sell_Q', 'sum')
        ).assign(profit=lambda x: x['total_sell'] - x['total_buy'])
        
        # Determina si el inventario es positivo y redondea el porcentaje de ganancia
        account_summary['invent'] = account_summary['total_buy_Q'] - account_summary['total_sell_Q']
        account_summary['is_invent_positive'] = account_summary['invent'] > 0
        account_summary['profit_perc'] = account_summary.apply(
            lambda row: (row['profit'] / row['total_buy'] * 100) if row['total_buy'] > 0 else 0, axis=1
        )
        account_summary['profit_perc'] = round(account_summary['profit_perc'], 1)
        account_summary['pair_tr'] = token
        
        summaries.append(account_summary)
        
    final_summary = pd.concat(summaries).reset_index()
    final_summary_sorted = final_summary.sort_values(by='profit_perc', ascending=False)
    
    return final_summary_sorted

In [3]:
# Carga el DataFrame
df = pd.read_csv(file_path, encoding='utf-8')

# Preprocesamiento
df['date'] = pd.to_datetime(df['date'])
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.time
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
df['value'] = df['value'].round(2)
df['buy'] = df.apply(lambda row: row['value'] if row['type'] == 'BUY' else 0, axis=1)
df['sell'] = df.apply(lambda row: row['value'] if row['type'] == 'SELL' else 0, axis=1)

# Nota de corte para valorar la rentabilidad  en variable NOTADE CORTE

In [4]:
# Procesa el DataFrame por cada 'pair_id_token'
final_summary_sorted = process_data_for_each_token(df)

In [5]:
# Suponiendo que 'final_summary_sorted' es tu DataFrame final con todas las columnas calculadas

# Define el nuevo orden de las columnas
new_column_order = [
    'account', 'pair_tr','total_buy_Q',
    'total_sell_Q', 'invent', 'is_invent_positive', 'total_buy', 'total_sell', 'profit','profit_perc'
]

# Reorganiza las columnas del DataFrame según el nuevo orden
top_summary = final_summary_sorted[new_column_order]

In [6]:
# Set display format for float
pd.set_option('display.float_format', lambda x: '%.1f' % x)
top_summary

,account,pair_tr,total_buy_Q,total_sell_Q,invent,is_invent_positive,total_buy,total_sell,profit,profit_perc
13671,0x4c4b357cf47e57fd4fabc7e7ee31044674bcb0d9,0x6514355c70d3be0ee0596883d5f09a1387a2b856,49825.0,5192000049825.0,-5192000000000.0,False,0.0,39.0,39.0,389800.0
26658,0x4c4b357cf47e57fd4fabc7e7ee31044674bcb0d9,0xbdb0915e09579a7a9a0921fbdaff9b8186fc207e,52364.0,5192000052364.0,-5192000000000.0,False,0.0,38.4,38.4,384200.0
10174,0x4c4b357cf47e57fd4fabc7e7ee31044674bcb0d9,0x578acb6e75a13761d70833b0bbdc39f170e52269,73395.0,5192000000000.0,-5191999926605.0,False,0.1,63.2,63.1,63100.0
29111,0x4c4b357cf47e57fd4fabc7e7ee31044674bcb0d9,0xd2150e23e3a9e128715a4be84edf8849bc329e67,64875.0,5192000000000.0,-5191999935125.0,False,0.1,55.6,55.5,55540.0
21417,0x4c4b357cf47e57fd4fabc7e7ee31044674bcb0d9,0x95c39d0eb1098528e7682b4880d489af1923e464,89800.0,5192000000000.0,-5191999910200.0,False,0.2,78.2,78.0,43338.9
...,...,...,...,...,...,...,...,...,...,...
13789,0xdeb804f640aa4596e081a994c184b8db79861e65,0x6514355c70d3be0ee0596883d5f09a1387a2b856,34211.0,0.0,34211.0,True,0.0,0.0,-0.0,-100.0
13788,0xddde4437ec5aa270ca05d6d2719114deb03c76ed,0x6514355c70d3be0ee0596883d5f09a1387a2b856,34405.0,0.0,34405.0,True,0.0,0.0,-0.0,-100.0
13787,0xddb9927ec6695363a8bfbe3fcfbf6369b3f89730,0x6514355c70d3be0ee0596883d5f09a1387a2b856,1405497.0,0.0,1405497.0,True,1.0,0.0,-1.0,-100.0
13786,0xdbc6db67cdba9c4eaf105c2d38af11032435edbf,0x6514355c70d3be0ee0596883d5f09a1387a2b856,189047.0,0.0,189047.0,True,0.0,0.0,-0.0,-100.0


In [19]:
# Filtrar 'top_summary' para incluir solo las filas donde 'is_invent_positive' es True Y 'profit_perc' es mayor que 50

NOTADECORTE = 50

top_summary_desnatado = top_summary[
    (top_summary['is_invent_positive'] == True) &
    (top_summary['profit_perc'] > NOTADECORTE) &
    (top_summary['total_buy'] >= 0.1)
]

top_summary_desnatado

,account,pair_tr,total_buy_Q,total_sell_Q,invent,is_invent_positive,total_buy,total_sell,profit,profit_perc
13840,0x0ce9cce19d9bde936dccd1d4855e5e8a6f0ea24c,0x6551157dbc11e25fae066114e5afeb180a944776,161376.0,158312.0,3064.0,True,0.2,3.2,3.0,1589.5
24775,0xf9a536581d125191415f2583833bf26b03a60dae,0xafe1ae67d202cda3c99669607358607e96cec268,328832.0,326753.0,2079.0,True,0.3,4.3,4.0,1137.1
25667,0x2a332a1a7bfee72001ea8ca2ecc56d9fa9fd6084,0xb810e19257115e370bdca7bbe5a97e07c5787666,11197.0,10307.0,890.0,True,0.2,2.9,2.7,1112.5
13981,0x6ed7b2822e18281408972f83f936cc95e31496a7,0x6551157dbc11e25fae066114e5afeb180a944776,171313.0,162309.0,9004.0,True,0.3,3.4,3.1,882.9
27886,0xe782c03212b034fbdd925fcff7d57cf964be231b,0xcc2938a163160a9040e22cb18fea65cf5d7b8371,72069.0,69252.0,2817.0,True,0.1,1.3,1.2,850.0
...,...,...,...,...,...,...,...,...,...,...
24080,0xc6e053d5067d8324e94d6290ffa81fb3bf29fcba,0xac92a1560b533bfc708af64e989b9ff08554f614,338133.0,301463.0,36670.0,True,0.8,1.2,0.4,50.6
28755,0x100fe1a29c6fc768774f43be23d0b8fa4d420b29,0xd1fc68dc1e961c8f86d87ab21d2b187753b26dba,67735.0,63726.0,4009.0,True,0.8,1.2,0.4,50.6
26268,0xe71d4ffde67403d74b659e4face2fdbad1a7a0a2,0xbbd6abce2937ccd2df41cb24393286ac24349df1,411600.0,0.0,411600.0,True,30.0,45.1,15.1,50.3
22195,0x70b48dbe14a174ad1a828fd2e581a4d327d3a97f,0x9875b31a4e9accbaa16cfb4b3cd25079db8c0b69,240594.0,228368.0,12226.0,True,2.0,3.0,1.0,50.2


In [20]:
# Grabación del fichero top traders con rendimiento mayor a NOTADECORTE con tokens creado en los últimos tres meses
file_path = 'hist_output/top_summary_desnatado.csv'

# Writing the DataFrame to a CSV file without the index column
df.to_csv(file_path, index=False)

print("DataFrame top_traders_agreg50_filtro_Q has been written to CSV successfully in folder hist_output")

DataFrame top_traders_agreg50_filtro_Q has been written to CSV successfully in folder hist_output
